<h4>TextCNN</h4>

<h5>Imports</h5>

In [1]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle

2023-04-22 21:25:57.179712: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-22 21:25:57.278157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 21:25:58.104278: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_names=['malicious','normal']
for name in file_names:
    data=[]
    with jsonlines.open('dataset/'+name+'.txt') as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data.append({'request':{'method':'POST', 'uri':line['request']['uri'], 'body':line['request']['body'],'headers':line['request']['headers']},'metadata':line['metdata']})
            else:
                data.append({'request':{'method':'GET', 'uri':line['request']['uri'], 'headers':line['request']['headers']},'metadata':line['metdata']})
    with jsonlines.open('dataset/'+name+'_clean.txt', mode='w') as writer:
        writer.write_all(data)

<h4>Load and create datasets</h4>

In [3]:
file_names = ['dataset/malicious_clean.txt' ,'dataset/normal_clean.txt']
data={}
for file in file_names:
    data[file]=[]
    with jsonlines.open(file) as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data[file].append('POST'+' '+line['request']['uri']+' '+line['request']['body']+' '+json.dumps(line['request']['headers']))
            else:
                data[file].append('GET'+' '+line['request']['uri']+' '+json.dumps(line['request']['headers']))
                
                
normal = data[file_names[1]]
malicious = data[file_names[0]]
normal_part1 = normal[180000:]
normal_part2 = normal[:180000]

train_examples = normal_part2
test_examples = normal_part1+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal_part1)
test_labels.extend([1] * len(malicious))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))


def preprocess_text(text, label):
    text = tf.strings.lower(text)
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())

2023-04-22 21:26:16.088135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-22 21:26:16.271871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-22 21:26:16.271919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-22 21:26:16.274244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-22 21:26:16.274285: I tensorflow/compile

b'a  / a / a / a  { "a" = "a" }   { "a-a": "a", "a": "a / a,* / *", "a-a": "a, a, a", "a-a": "a / n.n  ( a a n.n; x; x )  a / n.n  ( a, a a )  a / n.n.n.n a / n.n", "a": "a = x; a = x" } ' 0
b'post  / rest / products / reviews  { "id" = "ajgoztlb" }   { "accept-language": "de", "accept": "text / css,* / *", "accept-encoding": "gzip, deflate, br", "user-agent": "mozilla / 5.0  ( windows nt 10.0; win64; x64 )  applewebkit / 537.36  ( khtml, like gecko )  chrome / 72.0.3626.121 safari / 537.36", "cookie": "phpsessid = 33ptq3esqvyehncdmubiygychvpzr7gw; continuecode = 8dywbdjgirzm76ncpx2hf8kbrclgrd76" } ' 0


2023-04-22 21:26:19.186871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]
2023-04-22 21:26:19.214642: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]


<h4>Vocabularies</h4>

In [21]:

vectorization = TextVectorization(
    max_tokens=10000,
    output_mode="int", 
    output_sequence_length=128,
    pad_to_max_tokens=True,
    standardize="lower"
)

vectorization_sub = TextVectorization(
    max_tokens=2000, 
    output_mode="int", 
    output_sequence_length=128,
    pad_to_max_tokens=True,
    standardize="lower"
)

vectorization.adapt(train.map(lambda x, y: x))
vectorization_sub.adapt(train_sub.map(lambda x, y: x))

2023-04-22 21:02:03.817092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]
2023-04-22 21:10:25.488554: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]


<h4>Save and load vectorization from disk</h4>

In [4]:
#pickle.dump({'config': vectorization.get_config(),'weights': vectorization.get_weights()}, open("vectorization.pkl", "wb"))

#pickle.dump({'config': vectorization_sub.get_config(), 'weights': vectorization_sub.get_weights()}, open("vectorization_sub.pkl", "wb"))


from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-04-22 21:26:19.285886: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-04-22 21:26:19.435844: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


<h4>Autoencoder</h4>

In [5]:
embedding_layer = tf.keras.layers.Embedding(1000, 4)
flatten_layer = tf.keras.layers.Flatten()

def autoencoder_processing(text, label):
    return  vectorization_sub(text), vectorization_sub(text)

train_auto = train_sub.map(autoencoder_processing)

train_auto = train_auto.batch(32)
# Define the autoencoder model
input_layer = tf.keras.layers.Input(shape=(128,))
encoder = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoder = tf.keras.layers.Dense(128, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(64, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(128, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(128, activation='relu')(decoder)
autoencoder = tf.keras.models.Model(input_layer, decoder)

# Compile and train the model
autoencoder.compile(optimizer='adam',  loss=losses.MeanSquaredError())
autoencoder.fit(train_auto, epochs=10)

Epoch 1/10


2023-04-22 21:26:19.765127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype int64
	 [[{{node Placeholder/_5}}]]
2023-04-22 21:26:21.631094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-22 21:26:21.636668: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f8a63bebc00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-22 21:26:21.636711: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-04-22 21:26:21.642368: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set 

5625/5625 [==============================] - 43s 7ms/step - loss: 65.2075
Epoch 2/10
5625/5625 [==============================] - 40s 7ms/step - loss: 45.3778
Epoch 3/10
5625/5625 [==============================] - 39s 7ms/step - loss: 40.7097
Epoch 4/10
5625/5625 [==============================] - 39s 7ms/step - loss: 37.9835
Epoch 5/10
5625/5625 [==============================] - 40s 7ms/step - loss: 36.1523
Epoch 6/10
5625/5625 [==============================] - 40s 7ms/step - loss: 34.5001
Epoch 7/10
5625/5625 [==============================] - 40s 7ms/step - loss: 33.1057
Epoch 8/10
5625/5625 [==============================] - 40s 7ms/step - loss: 32.2520
Epoch 9/10
5625/5625 [==============================] - 40s 7ms/step - loss: 32.4991
Epoch 10/10
5625/5625 [==============================] - 39s 7ms/step - loss: 30.8432


In [14]:
autoencoder.fit(train_auto, epochs=10)
autoencoder.save('autoencoder/my_model')
autoencoder = tf.keras.models.load_model('autoencoder/my_model')


Epoch 1/10
5625/5625 [==============================] - 41s 7ms/step - loss: 30.0103
Epoch 2/10
5625/5625 [==============================] - 40s 7ms/step - loss: 29.9568
Epoch 3/10
5625/5625 [==============================] - 40s 7ms/step - loss: 29.4605
Epoch 4/10
5625/5625 [==============================] - 39s 7ms/step - loss: 28.9400
Epoch 5/10
5625/5625 [==============================] - 40s 7ms/step - loss: 28.7682
Epoch 6/10
5625/5625 [==============================] - 39s 7ms/step - loss: 27.6316
Epoch 7/10
5625/5625 [==============================] - 40s 7ms/step - loss: 28.1521
Epoch 8/10
5625/5625 [==============================] - 40s 7ms/step - loss: 27.0199
Epoch 9/10
5625/5625 [==============================] - 40s 7ms/step - loss: 28.6628
Epoch 10/10
5625/5625 [==============================] - 41s 7ms/step - loss: 27.8188
INFO:tensorflow:Assets written to: autoencoder/my_model/assets


In [15]:
def autoencoder_processing(text, label):
    return  vectorization_sub(text), label
test_auto = test_sub.map(autoencoder_processing)
test_auto = test_auto.batch(32)
reconsturctions =  autoencoder.predict(test_auto)

   8/3685 [..............................] - ETA: 28s 

2023-04-22 21:50:17.787743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64
	 [[{{node Placeholder/_7}}]]


3685/3685 [==============================] - 24s 6ms/step


In [16]:
def get_data(x, y):
  return x

# Use the map() function to extract the labels
dataset_tmp = test_auto.map(get_data)

# Convert the labels dataset to a NumPy array
tmp = np.concatenate(list(dataset_tmp.as_numpy_iterator()))
mse = tf.keras.losses.mean_squared_error(tmp, reconsturctions)

2023-04-22 21:50:42.582541: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype string
	 [[{{node Placeholder/_6}}]]


In [17]:
mse

<tf.Tensor: shape=(117899,), dtype=float32, numpy=
array([ 9.091696, 31.913082,  8.286144, ..., 11.511646, 76.687546,
       17.263046], dtype=float32)>

In [20]:
vectorization_sub.get_vocabulary()

['',
 '[UNK]',
 'a',
 '/',
 '=',
 '"a-a":',
 '"a',
 'n.n',
 '(',
 ')',
 '"a":',
 'a,',
 'n.n",',
 'x;',
 '{',
 '}',
 'n.n.n.n',
 'x"',
 'a,a',
 'x',
 'n.n;',
 'a,*',
 '*",',
 '*;a',
 '"a",',
 'a;a',
 'a",',
 '"a,',
 'n.n,',
 '?',
 'n.n.n.n",',
 '"a-a,a;a',
 'a;',
 'n_n_n',
 'n.n,a',
 'n',
 '"a;a',
 'a&a',
 'a.a',
 '"*',
 '"a-a",',
 '"a-a,a-a",',
 'n&a',
 '"a":"a',
 'a.*',
 'x&a',
 'a","a":"a',
 'a"',
 'a","a":',
 'n,"a":"n","a":"n',
 'a","a":"a","a":"a"',
 '*',
 'a&a-a',
 "a'a",
 'n:x","a":"n-n-x:n:n.n',
 'n:x"',
 ',"a":"a"',
 '"a":n,"a":"a',
 '","a":"n-n-x:n:n.n',
 'a-a',
 'x@a.a&a',
 'a_a_a_a.a',
 'a","a":n,"a":"n","a":"n"',
 '.',
 '[',
 ']',
 '"a"',
 'a.',
 '"a":n,"a":"a","a":n',
 'a."',
 '"a":"x@a.a","a":"a","a":"a","a":',
 '&a',
 '"a":"x@a.a","a":"a"',
 '"x"',
 'a","a":"a"',
 'a:"',
 '"a":"n","a":"n","a":"n"',
 'a","x"',
 '"',
 'x&a-a',
 'a_a_a_a',
 'a_x_x.a',
 'a_x',
 'a:',
 '"a":"x","a":',
 'x_x',
 'x-x@a.a&a',
 'x.x@a.a&a',
 'x_x@a.a&a',
 'a_a_x.a',
 'a@a.a&a',
 'a.a.a',
 'a_x@